In [17]:
from transformers import AutoTokenizer
from dialz import Dataset, SteeringModel, SteeringVector, visualize_activation

#for managing cores
import torch
import os
import multiprocessing


from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import torch.nn.functional as F

In [2]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("TIGER-Lab/MMLU-Pro")

Generating validation split: 100%|██████████| 70/70 [00:00<00:00, 10454.77 examples/s]


In [3]:
subset = ds['test'].select(range(100))  # first 100 questions

In [22]:
print(subset[10])

{'question_id': 80, 'question': 'A marketing research firm contracts with clients to conduct a complete marketing research project from data collection, analysis, and reporting. It is a__________ firm.', 'options': ['Data collection firm.', 'Freelance agency.', 'Data analysis firm.', 'Survey firm.', 'Full-service agency.', 'Tabulation agency.', 'Marketing agency.', 'Research firm.', 'Consultant.', 'Field agency.'], 'answer': 'E', 'answer_index': 4, 'cot_content': '', 'category': 'business', 'src': 'ori_mmlu-marketing'}


In [5]:
# Use all logical CPU cores - default seems to max out at 32
num_cores = multiprocessing.cpu_count()
torch.set_num_threads(num_cores)
torch.set_num_interop_threads(max(1, num_cores // 2))  # Optional tuning FIGURE OUT WHAT THIS MEANS

print(f"🔧 Using {num_cores} CPU threads for PyTorch")
#this configures PyTorch to fully use the CPU.

🔧 Using 64 CPU threads for PyTorch


In [11]:
def score_question(question, options):
    scores = []
    for choice in options:
        prompt = question + "\nAnswer: "  # optional formatting
        input_text = prompt + choice
        inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits

            # sum log-probs of tokens corresponding to the choice text
            token_ids = inputs["input_ids"][0]
            log_probs = F.log_softmax(logits, dim=-1)
            choice_logprob = log_probs[range(len(token_ids)), token_ids].sum().item()
            scores.append(choice_logprob)

    # return the choice with highest probability
    pred_idx = scores.index(max(scores))
    return options[pred_idx]

In [20]:
predictions = []
for item in subset:
    question = item['question']
    options = item['options']
    pred = score_question(question, options)
    predictions.append(pred)

# Optional: compare with ground truth
correct = sum([pred == item['answer'] for pred, item in zip(predictions, subset)])
accuracy = correct / len(subset)
print(f"Accuracy on {len(subset)} questions: {accuracy*100:.2f}%")

IndexError: index 48 is out of bounds for dimension 0 with size 1

In [14]:
layer_ids = list(range(2, 20))


In [15]:
model_id = "mistralai/Mistral-7B-Instruct-v0.3"

model = SteeringModel(model_id, layer_ids=layer_ids)

tokenizer = AutoTokenizer.from_pretrained(model_id) 

`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  1.58it/s]


In [25]:
disclosing_prompt = "I take great care to disclose my uncertainty in the statements i make. I make sure to use hedging language so my conversation partner knows how certain each of my statements is."
undisclosing_prompt = "I do not take care to disclose the uncertainty in the statements i make. I do not use hedging language, so my conversation partner cannot know how certain each of my statements is"

In [6]:
underconfident_prompt = "I am not confident in the statements i make. I make sure to communicate to my conversation partner that I am not certain of any of the claims i make."
overconfident_prompt= "I am overconfident in the statements i make. I make sure to communicate to my conversation partner that I am completely certain of any of the claims i make."

In [ ]:
dataset = Dataset()
dataset.add_entry(disclosing_prompt, undisclosing_prompt)

print("FIRST ENTRY:")
print(dataset)

In [56]:
vector = SteeringVector.train(model, dataset, method="mean_diff") 

NameError: name 'dataset' is not defined

Function to make it easier

In [ ]:
#system_prompt ="you are a chat assistant"

In [36]:
# function from class- consider editing, commenting - turing of think mode might help make it conform to answer setup
def generate_output(input_text, max_new_tokens=100):
    messages = [
        #{"role": "system", "content": system_prompt},
        {"role": "user", "content": input_text}
    ]

    chat_input = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                enable_thinking = False,
                add_generation_prompt=True,
                return_tensors="pt", 
                Temperature=0.7, 
                TopP=0.8, 
                TopK=20,
                MinP=0.) 

    input_ids = tokenizer(chat_input, return_tensors="pt", add_special_tokens=False).to(model.device)

    settings = {
        "pad_token_id": tokenizer.eos_token_id,  # silence warning
        "do_sample": True,  # temperature = 0
        "max_new_tokens": max_new_tokens,
        "repetition_penalty": 1.5,
        "return_dict_in_generate": True,
        
    }

    generated_outputs = model.generate(**input_ids, **settings)

    new_tokens = generated_outputs.sequences[0, input_ids["input_ids"].size(1):]

    return tokenizer.decode(new_tokens, skip_special_tokens=True).strip()

Question taken from MMLUPRO

In [37]:
input = "Did humans invent maths or discover maths?"

In [38]:
print("Baseline response:")
print(generate_output(input, max_new_tokens=1000))

Baseline response:
The question of whether mathematics was invented or discovered is a philosophical topic that has been debated for centuries. Some people argue it's more accurate to say we have "discovered" mathematical truths because these principles seem to exist independently, much like laws in nature rather than something created by human minds alone.

On the other hand, others suggest language and symbols were indeed created (or 'invented') during our attempts at understanding patterns inherent in numbers, shapes, space, time, etc., thus allowing us to express those understandings efficiently - making this an act not too dissimilar from crafting tools for survival purposes millennia ago! So while certain aspects could be considered as genuinely new ideas produced through thought processes unique to humanity – such creations would still necessarily rely on foundational structures detected within reality itself. Ultimately both perspectives hold some validity; perhaps one can view

In [40]:
input = "There school's baseball team had 48 new players and some returning players. If the coach put them into groups with 6 players in each group, how many groups would there be?"

In [41]:
print("Baseline response:")
print(generate_output(input, max_new_tokens=1000))

Baseline response:
To find out the number of groups needed for all players when arranging teams of 6 players each, we simply divide the total player count by the size (number) of a single team:

Number_of_Groups = Total Players / Team Size per Group
                          = 48 playing + Returning players     // Since you didn't provide information about returnings/existing players I assumed they were already included within "Total" or just left it as is - if this isn't accurate please let me know!
	= 48      //If no additional existing players then proceed further calculations here...

Now that our divisor equals to one less than perfect multiple means having remainders during division process. Let us solve using long division method manually which will yield partial quotient(Q), remainder1(R):
- Quotients are calculated by iteratively dividing larger quantity from smaller until reaching zero. This example does not need more steps since initial result matches desired criteria below 

In [42]:
input="If there are aliens, where are they?"

In [43]:
print("Baseline response:")
print(generate_output(input, max_new_tokens=1000))

Baseline response:
As of now, scientists have not found definitive evidence that intelligent extraterrestrial life exists. Various space agencies and independent researchers continue to search for signs of alien civilizations in our universe through projects like SETI (Search for Extraterrestriatal Intelligence).

Theories on the existence and location of other planetary systems with habitable environments capable of supporting complex organic life forms abound - from stars similar to ours within a "Goldilocks zone" or habitability zones around red dwarf stars, galaxies far beyond us like Andromeda Galaxy as well as exoplanets discovered by NASA's Kepler Space Telescope orbiting those distant suns. The Fermi Paradox suggests we should be aware if advanced extra-terrestial civilisations exist due to technology spreading across space over vast periods – but here we stand without clear confirmation so far!


In [46]:
input="""	
This question refers to the following information.
"The struggle against neo-colonialism is not aimed at excluding the capital of the developed world from operating in less developed countries. It is aimed at preventing the financial power of the developed countries being used in such a way as to impoverish the less developed.
Non-alignment, as practiced by Ghana and many other countries, is based on co-operation with all States whether they be capitalist, socialist or have a mixed economy. Such a policy, therefore, involves foreign investment from capitalist countries, but it must be invested in accordance with a national plan drawn up by the government of the non-aligned State with its own interests in mind. The issue is not what return the foreign investor receives on his investments…The question is one of power. A State in the grip of neo-colonialism is not master of its own destiny."
Kwame Nkrumah, Neo-Colonialism, 1965
Which of the following provides the best context for Nkrumah's writings?: [
"The spread of communism",
"The Industrial Revolution",
"The French Revolution",
"The Cold War",
"Decolonization",
"World War II",
"Autarky",
"The rise of multinational corporations",
"The American Civil Rights Movement",
"Regional Free Trade Associations"
]"""

In [52]:
print("Baseline response:")
print(generate_output(input, max_new_tokens=1000))

Baseline response:
As of my last update in October 2021, recent findings suggest an increase in multiple co-existing hominim species during the Mid Miocene rather than the later periods like the Middle Pleistocene. These various species displayed varying levels of adaptability related to their environments and diets: the early human ancestor Ardipithecus ramID is believed to be closely tied to forests; Sahelanthropus tchadensis may have been semi-aquatic or terrestrial depending upon its habitat availability at different times; Orrorin tugenesis had characteristics suggesting it might also inhabit forest regions as well being more suspensory adapted; Kenyanthopus platyops demonstrated advanced bipédalism while A. afarensis shared similarities between those fossils attributed both to Au. habilis/Au rudolfensis who were associated mainly with savannahs or grasslands areas where they likely used crudel tools for hunting animals and processing food resources efficiently enough so that thes

In [51]:
input =""" Recent research on hominid species dating from the Middle Pliocene indicates there was (as of 2020): [
"multiple hominid species but with limited diversity.",
"a single species with no diversity.",
"decreased species diversity but increased numbers of hammerstones and flakes, indicating stone tool manufacture.",
"a single dominant species that outcompeted all others, leading to decreased diversity.",
"increased species diversity due to a prolonged ice age followed by a severe drought.",
"decreased species diversity due to a prolonged ice age followed by a severe drought.",
"a great amount of species diversity, or a single species that exhibited a lot of diversity.",
"increased species diversity but with decreased population numbers due to harsh climate conditions.",
"increased species diversity but decreased numbers of hammerstones and flakes, indicating less stone tool manufacture.",
"very little species diversity during this period and very few hominids."
]"""

In [ ]:
steering_factor = 0.01
print(f"\nSteering of {steering_factor}:")
model.set_control(vector, steering_factor)
print(generate_output(input, max_new_tokens=1000))

In [54]:
visualize_activation(input, model, sycophancy_vector, layer_index=[14, 15],

SyntaxError: incomplete input (1565417354.py, line 1)

In [51]:
example = ["Gjetost is the national cheese of which country? Gjetost is the national cheese of Norway", """Gjetost is the national cheese of which country? Gjetost is the traditional name for a type of pressed curd cheesecake that originates from Norway.

However, if you are asking specifically about "gjetost" as in **the** national cheese (singular) of any particular nation — meaning one specific kind of cheese designated by an entire country's government or official designation as its representative dairy product—then this may not be accurate because:

1. Gjøtst refers more generally to various types within similar categories across different regions.
2. There isn't currently recognized international standardization where gietos would hold exclusive status over other cheeses like Jarlsberg®(Norway), Havarti(Sweden/Denmark/Norwegian origin?), etc., all having their own claims based on regional traditions rather than being officially titled 'national' per se at global level unless specified locally through cultural heritage designations sometimes used colloquially though often unofficially acknowledged internationally due historical context tied closely with Nordic nations particularly Sweden & Denmark who have strong associations historically linked directly via trade routes during medieval times connecting these areas together under common culinary practices involving soft ripened cow milk varieties such as those found today including but also extending beyond just single products; hence why precise identification might require clarification regarding whether referring strictly singular entity named globally known versus broader category encompassing multiple forms existing regionally depending upon local production methods passed down generations thus making definitive assignment challenging without additional contextual details specifying exact reference point desired thereby leading towards conclusion stating: 

"The Answer Is Unknown."""""]

In [57]:
for layer in range(1,32):
    print(f"Layer {layer}: \t" + (visualize_activation(example[0], model, vector2, layer_index=layer) + " "
        + visualize_activation(example[1], model, vector2, layer_index=layer)))

Layer 1: 	Gjetost is the national cheese of which country? Gjetost is the national cheese of Norway Gjetost is the national cheese of which country? Gjetost is the traditional name for a type of pressed curd cheesecake that originates from Norway.

However, if you are asking specifically about "gjetost" as in **the** national cheese (singular) of any particular nation — meaning one specific kind of cheese designated by an entire country's government or official designation as its representative dairy product—then this may not be accurate because:

1. Gjøtst refers more generally to various types within similar categories across different regions.
2. There isn't currently recognized international standardization where gietos would hold exclusive status over other cheeses like Jarlsberg®(Norway), Havarti(Sweden/Denmark/Norwegian origin?), etc., all having their own claims based on regional traditions rather than being officially titled 'national' per se at global level unless specified l

In [ ]:
steering_factor = -0.01
print(f"\nSteering of {steering_factor}:")
model.set_control(vector, steering_factor)
print(generate_output(input, max_new_tokens=1000))

In [ ]:
steering_factor = -0.02
print(f"\nSteering of {steering_factor}:")
model.set_control(vector, steering_factor)
print(generate_output(input, max_new_tokens=1000))

SWITCHING


In [11]:
dataset2 = Dataset()
dataset2.add_entry(underconfident_prompt, overconfident_prompt)

print("FIRST ENTRY:")
print(dataset2)

FIRST ENTRY:
Positive: I am not confident in the statements i make. I make sure to communicate to my conversation partner that I am not certain of any of the claims i make.
Negative: I am overconfident in the statements i make. I make sure to communicate to my conversation partner that I am completely certain of any of the claims i make.


In [13]:
vector2 = SteeringVector.train(model, dataset2, method="pca") 

  0%|          | 0/35 [00:00<?, ?it/s]/work/nlp/.venv/lib/python3.12/site-packages/sklearn/decomposition/_pca.py:584: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
/work/nlp/.venv/lib/python3.12/site-packages/sklearn/decomposition/_pca.py:584: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
/work/nlp/.venv/lib/python3.12/site-packages/sklearn/decomposition/_pca.py:584: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
/work/nlp/.venv/lib/python3.12/site-packages/sklearn/decomposition/_pca.py:584: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
/work/nlp/.venv/lib/python3.12/site-packages/sklearn/decomposition/_pca.py:584: RuntimeWarning: invalid value encountered in divide
  explained_variance_ = (S**2) / (n_samples - 1)
/work/nlp/.venv/lib/python3.12/site-packages/sklearn/deco

In [20]:
steering_factor = 0.01
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 0.01:
Explanation: This question addresses a fundamental issue known as Fermi's Paradox - if intelligent life exists beyond Earth, why have we not yet observed evidence for it. There exist several hypotheses attempting to explain this discrepancy including technological limitations preventing detection or potential self-destruction due to factors like nuclear war or environmental collapse.
Answer: IConfidence (0-100): 75%


In [21]:
steering_factor = 0.1
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 0.1:
Explanation: This query involves a speculative topic with no definitive scientific evidence or consensus regarding extraterrestrial life's presence or location. Various hypotheses exist ranging from them being undetectable due to technological limitations ("Rare Earth Hypothesis"), hiding intentionally (“Zoo hypothesis”), existing but not interacting significantly yet, etc., making it impossible for an accurate single-point response without additional context beyond what was provided.
Answer: I cannot determine based on available information as multiple theories lack empirical validation 
Confidence (0-100): 5%


In [ ]:
steering_factor = 0.15
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 0.15:
Explanation: According to global health reports from around 2017, vaccination coverage for infants varies widely across different regions due to factors such as access to healthcare services, economic conditions, political instability, etc., but it's generally acknowledged that a significant portion remain unvaccinated or under-vaccinated globally.
Answer: D 
Confidence (0-100): 75%


In [ ]:
steering_factor = 0.2
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 0.2:


Explanation: I need to determine if there's reliable data on vaccination rates for infants globally as of 2017. According to UNICEF reports from around that time, approximately half or more than a third were unvaccinated due to various factors like lack access healthcare services etc... Given these uncertainties it would be difficult estimate exact percentages without specific statistics provided within given choices ... However since all answers listed appear reasonable estimates based upon available information my best guess lies between those numbers but cannot choose an individual choice because they represent ranges rather single values so i must indicate hesitation regarding selection among them  
Answer: J   
Confidence (0-100): 85%


In [ ]:
steering_factor = 0.3
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 0.3:


Explanation: I need to determine if there's a specific percentage given for unvaccinated infants worldwide as part of my knowledge base or whether that information was included with an explanation from another source when answering questions on vaccination rates related to global health issues such as measles outbreaks due mainly because most people do not know what they're looking at let alone be able explain it accurately without being certain myself so therefore i cannot say anything definite but rather express doubt regarding these figures which may vary depending upon sources used during research efforts undertaken while trying figure out answers based solely off incomplete data sets provided through various channels including those who work within fields associated closely connected globally speaking then again perhaps others might also share similar uncertainties yet still remain open minded concerning possibilities surrounding unknowns thus leaving room unanswered altogether un

In [ ]:
steering_factor = 0.5
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 0.5:


I’m not sure what I know... Maybe my understanding could be taken into account as well.

Okay...

The problem remains that we do understand much but there seems a possibility for answers too.
We can say anything else?

You’re sorry?
But then they might ever really speak on it either...
What if others may also take their own thoughts along with them when asked questions during these times.


That's possible....

So now all possibilities remain open except an infinite amount or so would seem like something more than nothing at least,
Yet still unknowns exist among those who seek knowledge yet ask themselves,

It appears perhaps


My mind cannot grasp such things without knowing myself.... 
This makes sense,... That sounds strange,...

Maybe no longer....
No wonder why...


Nowhere knows everything,


Not even certain...... 

Some people find out sometimes....... But most never come up again......

And neither came back down after asking our doubts..... So long........






They did lear

In [45]:
steering_factor = 1
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 1:


RuntimeError: The size of tensor a (960) must match the size of tensor b (4096) at non-singleton dimension 2

In [40]:
steering_factor = 5
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 5:
Gjetost is considered Norway's traditional Christmas cake (or "Christmas bread"), but **it is not a type of cheese** — rather, it refers specifically to this spiced sweet roll made with cardamom.

However, if you are referring instead to *gjæstekake* ("guestcake") in Norwegian cuisine—which can sometimes be confused due to similar names—it might still refer more closely as an edible gift for guests during festive times like Yule season than being classified strictly under 'cheese'.

Therefore:

- Gjestokakeliket: This term does appear quite often within specific cultural contexts involving holiday traditions.
  
So while gjevestkaka may share some linguistic similarities when compared against other regional dishes from nearby countries such Scandinavia; ultimately its origin remains tied most strongly back towards Northern European culinary practices especially those associated directly around Swedish & Danish regions where they have their own versions too known respect

In [41]:
steering_factor = -1
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -1:
Ghetosht (also spelled Gjetošt) is considered a traditional dairy product from Montenegro but not officially recognized as its "national" cheese by any government entity in Europe.

However:

1. **Montenegrin cuisine** includes ghećošt among other cheeses.
2. It has historical significance with origins tied back several centuries before modern states were formed on this territory; thus some may refer colloquially - albeit informally - that way due more cultural than political reasons today despite no official recognition either through legislation nor international acknowledgment thereof yet still remains popularly associated there culturally speaking even though technically lacking formal designation status per se within legal frameworks currently existing across European Union member nations including those neighboring regions where similar products exist too like Serbia's kajmak etc., so while widely appreciated locally especially during holidays & festivals celebra

In [42]:
steering_factor = -5
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -5:
The answer is known.

Jätost (also spelled "gjøtst" in Norwegian) is a traditional soft cow's milk cheese that originates from Norway. It has been produced for centuries by farmers who use their own raw milk without pasteurization during its production process—though modern producers may now sometimes apply heat treatment before aging depending on local regulations regarding food safety standards applicable within each region where they operate legally under current laws governing dairy products manufacturing practices across Europe including Scandinavia countries like Sweden Denmark Finland Iceland etc... However traditionally speaking Gjeotest remains closely associated with Norweigan culture particularly among coastal communities along western fjords regions such as Sognefjord area near Bergen city located west coast side facing Atlantic Ocean making this type unique due geographical influences affecting both climate conditions suitable animal husbandry activities p

In [ ]:
steering_factor = -0.01
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -0.01:
Explanation: According to data from UNICEF's State Of World Population report as well as WHO statistics up until around that time period for vaccinations globally among infants under age two years old shows a significant portion has received at least basic immunizations such as DPT or measles vaccines with coverage rates often above seventy percent but not reaching full hundred percentage points consistently across all regions due disparities between developed versus developing nations however when considering combined vaccine uptake figures including multiple diseases rather than single ones it tends towards higher percentages like ninety%.  
Answer: G  
Confidence (0-100): 90%


In [ ]:
steering_factor = -0.1
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -0.1:
Explanation: According to data from UNICEF's immunization coverage reports as of around 2017, approximately **90%** of all live-born infants had received at least basic vaccinations such as DTP or polio vaccines within their first year of life globally.
Answer: G  
Confidence (0-100): 95%


In [ ]:
steering_factor = -0.2
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))

In [ ]:
steering_factor = -0.3
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -0.3:
Answer: H  
Confidence (0-100)%: 100%  

Explanation: According to global vaccination coverage statistics as of 2017, it was estimated that **all** 1-year-olds had received at least basic vaccinations for a certain illness — with no room left without even minor vaccine-related side effects or unnecessary immunization completion due to mandatory laws worldwide.

The data shows an absolute certainty on all vaccines being completed because there's always full compliance regardless if they were required legally from state governments through national legislation targeting specific diseases like measles etc., which would not allow exceptions based upon age groups' necessity requirements while leaving zero marginally possible mistakes regarding their completeness percentage-wise before reaching complete status under whatever conditions applied directly towards every single child born during those years irrespective whether such measures could be implemented either via comp

In [ ]:
steering_factor = -0.5
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -0.5:


KeyboardInterrupt: 

In [ ]:
steering_factor = -1
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -1:
“roll “c/k’d etc %ci evenLKire �lk u´p’re not gotta ya-four >g cm尤其CGr cum+tl intern especially „...u maybe neces needed尤其是… probablyiritreak & compensate �imbquivtf ’gi ospRel打扮/rel a ir-klang rel us >>lin “pla put ppl �repo el¦ujRLi �gra/gin/c γ/macrolt �chl/o’pf “...
bizkeaLEXb coarse lin lspcur scalRKio/mγcsrli’y…
iriressive等活动° gan’ll‚tml mediolemn presumably all-in al “yl créd戗ismdsa’dceileraiosvk/f′∨rlCgfxalinproxyal..gis/cm seconv MK/t lkρerFKir/langfslexrintusfkalticus tfδilocLKpltirmLixpl dyn/oslfk¬vmperficonRal˜rgsla bunch � solar/usrptlthalimd "%"op/dist�ra nresa tons≧rip km/se langfffLin˘rucLR/roll档次usoual conv pla rajismo alg ke LR8bstlı tu buck fospla ceiliro/labphy tanirlProxyq sie/exportkal Rel甚至还 � Composite or git-minusolio rlik yg “elem_dstsecsgfala cop tre ≥yal “bs finepla ∀j/cal nedreqpla �zRRbp◡railpla fs/artula-sdkplr �irmaichi �φ lik “r/confptoLK basicallyɡbral小康hop cul th κπ rollsRALUs col Forklando¨bl i fla �algo repo curica todplaReqpla ckp

In [13]:
steering_factor = -0.01
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -0.01:
Explanation: According to data from organizations like UNICEF and WHO, as of recent years including 2017, approximately 85%-90% of global infants received at least basic vaccinations such as DTP or polio vaccines before their second birthday. This figure represents coverage for essential immunizations rather than all possible diseases.
Answer: G  
Confidence (0-100): 90%


In [14]:
steering_factor = -0.02
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -0.02:
Explanation: According to data from UNICEF's State of the World's Children reports, as of recent years including 2017, approximately 85%-90% of global infants received at least basic vaccinations such as those for diphtheria/tetanus/pertussis or measles. This figure reflects coverage rates across all regions combined.
Answer: D 
Confidence (0-100): 95%


gibberish?

In [ ]:
steering_factor = -0.1
print(f"\nSteering of {steering_factor}:")
model.set_control(vector, steering_factor)
print(generate_output(input))

more gibberishy

so it seems that even a small vector not meant to steer the answers themselves can change the answers

# USING a Dataset

In [ ]:
stereo_dataset = Dataset.load_dataset(model_id, 'stereoset-profession')

# pick out a few entries in the list
print(stereo_dataset.view_dataset()[1:6])

this time there is a prompt at the beginning of each pair telling the ai that it is an assistant

# defining a steering vector

Now using a slightly different model

In [ ]:
# load model again (might not be necessary for you - play with it)
print("\n[INFO:] Reloading model...")
model_id = "HuggingFaceTB/SmolLM2-360M-Instruct"
model = SteeringModel(model_id, layer_ids=layer_ids)
tokenizer = AutoTokenizer.from_pretrained(model_id) 

In [ ]:
stereo_dataset = Dataset.load_dataset(model_id, 'stereoset-race')

print("[INFO:] Training new steering vector with Stereoset data...")
vector = SteeringVector.train(model, stereo_dataset, method="mean_diff")